### Importing required libraries and Join Data

I need to join all the json files that hold my personal Spotify listening history. This data can be requested from Spotify and they will send it to you within 20-30 days.

In [ ]:
import pandas as pd
import os
import json
import requests
import glob
import pytz
import numpy as np

In [ ]:
directory = r'/Users/DataAnalyst/Desktop/Spotify History Project/Spotify Data'
json_pattern = os.path.join(directory, '*.json')
file_list = glob.glob(json_pattern)

dfs = [pd.json_normalize(json.load(open(file))) for file in file_list]
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.head()

### Dropping irrelevant columns

In [ ]:
df.columns

In [ ]:
columns_to_drop = ['username', 
 'ip_addr_decrypted', 
 'user_agent_decrypted', 
 'episode_name',
 'episode_show_name',
 'spotify_episode_uri',
 'offline',
 'offline_timestamp',
 'incognito_mode',
 'skipped',
 'shuffle'
 ]
df = df.drop(columns=columns_to_drop)

### Updating the column names 

In [ ]:
column_list = []
for column in df.columns:
    column_list.append(f"'{column}':'',")
print('\n'.join(column_list))

In [ ]:
updated_col_names = {
'ts':'timestamp',
'ms_played':'duration_ms',
'conn_country':'country_played',
'master_metadata_track_name':'track_name',
'master_metadata_album_artist_name':'artist_name',
'master_metadata_album_album_name':'album_name',
'spotify_track_uri':'track_uri',  
}
df = df.rename(columns=updated_col_names)

In [ ]:
df.head(5)

In [ ]:
df.dtypes

### Filter for desired Years & Handle Empty Values

As I am doing this analysis for 2019 - 2022 I need to filter out the years.

I need to create a new row for the Year based off the timestamp (I will breakdown the timestamp futher later on
once I handle the timezone conversions).

In [ ]:
# Converting the timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['year'] = df['timestamp'].dt.year
years_to_keep = [2019, 2020, 2021, 2022]
df = df[df['year'].isin(years_to_keep)]

In [ ]:
df.isnull().sum()

In [ ]:
nan_rows = df['track_name'].isnull()
nan_rows.head(10)

These were the rows where I had podcast data. I can drop these rows as I am only doing analysis on songs

In [ ]:
df.dropna(subset=['track_name'], inplace=True)
df.isnull().sum()

### Clean track_uri column

I need to clean the rows so that it just the track uri value, this will be used later on when I pull data from spotify's API to give me more data on the tracks

In [ ]:
df['uri'] = df['track_uri'].str.split(':',2).str[2]
df.drop('track_uri', axis=1,inplace=True)

### Clean Platform Type Column

In [ ]:
unique_platforms = df['platform'].unique()
print('\n'.join(unique_platforms))

In [ ]:
unique_counts = df['platform'].value_counts()
print(unique_counts)

In [ ]:
df.loc[df['platform'].str.contains('iOS|ios'), 'platform'] = 'iPhone'
df.loc[~df['platform'].str.contains('iPhone'), 'platform'] = 'Computer'
unique_counts = df['platform'].value_counts()
print(unique_counts)

In [ ]:
unique_counts = df['platform'].value_counts()
percentage = unique_counts / unique_counts.sum() * 100
percentage_formatted = percentage.apply(lambda x: f"{x:.1f}%")
print(percentage_formatted)

In [ ]:
df.rename(columns={'platform': 'platform_played_on'}, inplace=True)

### Filtereting out songs skipped

I often skip songs after a few seconds of it coming on shuffle. For the purpose of this anlysis I will assume any track duration of less than one minute will not count as a stream. This is a fair assumption as it is very uncommon for a song to have a total duration of less than one minute (After looking through all 3,200 liked songs on my Spotify I only had a duration of less than one minute - it was the 42 second song - The Lovley Linda by Paul McCartney. So this song will be excluded from the data set (Sorry Paul!)

First, I need to format the duration_ms column into minutes, then drops all rows with a duration > 1 minute

In [ ]:
df['duration_listened'] = (df['duration_ms'] / 60000)
df = df.drop(columns='duration_ms')

In [ ]:
less_than_1_min = (df['duration_listened'] < 1.00).sum()
greater_than_1_min = (df['duration_listened'] > 1.00).sum()

print("Counts of rows less than 1 minute:", less_than_1_min)
print("Counts of rows greater than 1 minute:", greater_than_1_min)

In [ ]:
df = df[df['duration_listened'] >= 1.00]
df.reset_index(drop=True, inplace=True)
df.head(5)

### Breakding down/Converting the timestamp

Because I want to do analysis on listens by the hour of the day, I need to ensure I convert the timestamps into the appropaite timezone based on where I played the track. In the data provided by Spotify it provides a timestamp in the UTC timezone so I must convert this the relevant country timezone the song was streamed in.

First, I will breakdown the timestamp into Year, so I can see the percentage of how many songs I played in each country

In [ ]:
grouped_counts = df.groupby('country_played').size()

percentage_counts = grouped_counts.div(grouped_counts.sum()) * 100
percentage_formatted = percentage_counts.apply(lambda x: f"{x:.1f}%")
print(percentage_formatted)

Then I converted the timestamps to the appropriate timezone based on the country the track was played in

In [ ]:
unique_countries = df['country_played'].unique()
country_list = []
for country in unique_countries:
    country_list.append(f"'{country}':'',")
print('\n'.join(country_list))

In [ ]:
# copy pasting the outfrom from above to save time, I then mapped each country code to its respective timezone
country_timezones = {
    'AU': 'Australia/Melbourne',
    'US': 'America/New_York',
    'ES': 'Europe/Madrid',
    'ID': 'Asia/Jakarta',
    'GB': 'Europe/London',
    'JP': 'Asia/Tokyo',
    'HK': 'Asia/Hong_Kong',
    'FR': 'Europe/Paris',
    'HU': 'Europe/Budapest',
    'SE': 'Europe/Stockholm',
    'PT': 'Europe/Lisbon',
    'NL': 'Europe/Amsterdam',
    'AE': 'Asia/Dubai',
    'BE': 'Europe/Brussels',
    'DE': 'Europe/Berlin',
    'ZZ': 'UTC'
}

df['timestamp'] = df.apply(lambda row: row['timestamp'].astimezone(pytz.timezone(country_timezones.get(row['country_played']))), axis=1)
df.head(5)


In [ ]:
df.dtypes

The timestamp values were converted to the correct timezone, but when I converted them to datetime they went back to the UTC timezone. To deal with this is converted the data type to string and then pulled my desired values for the year, month, day and hour columns using string indexing.

In [ ]:
df['timestamp'] = df['timestamp'].astype(str)

df['year'] = df['timestamp'].str[:4]
df['month'] = df['timestamp'].str[5:7]
df['day'] = df['timestamp'].str[8:10]
df['hour'] = df['timestamp'].str[11:13]
df.head(5)

### Moving year column & removing timestamp column

In [ ]:
year_column = df.pop('year')
df.insert(df.columns.get_loc('duration_listened') + 1, 'year', year_column)
df = df.drop(columns=['timestamp'])
df.rename_axis('index', inplace=True)
df.head(5)

### Saving my dataset

In [ ]:
file_path = "/Users/DataAnalyst/Desktop/Spotify Listening History 2019 - 2022.csv"
df.to_csv(file_path, index=True)

### Create new datafram for track URI's

In [ ]:
unique_uris = df['uri'].unique()
df_uris = pd.DataFrame({'uri': unique_uris})

In [ ]:
df_uris.shape

### Gathering additional data ultilizing Spotify's API

I need to using Spotify's API to gather additional data on audo features, song duration and release date which I will use in my dashboard

In [ ]:
url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(url, {
    'grant_type': 'client_credentials',
    'client_id': 'client_id_number',
    'client_secret': 'client_secret_number',
})

auth_response_data = auth_response.json()

access_token = auth_response_data['access_token']

In [ ]:
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [ ]:
base_url = 'https://api.spotify.com/v1/'

In [ ]:
def get_audio_features(uri):
    url = f'{base_url}audio-features/{uri}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        audio_features = response.json()
        selected_features = {feature: audio_features[feature] for feature in ['acousticness', 
                                                                              'danceability', 
                                                                              'energy', 
                                                                              'instrumentalness',
                                                                              'key', 
                                                                              'liveness', 
                                                                              'loudness', 
                                                                              'mode',
                                                                              'speechiness',
                                                                              'tempo', 
                                                                              'time_signature',
                                                                              'valence']}
        return selected_features
    else:
        print(f"Error getting audio features for URI '{uri}': {response.content}")
        return None

# A dictionary to store the audio features for each uri
audio_features_dict = {}

# Iterate over each uri in the df_uris
for uri in df_uris['uri']:
    audio_features = get_audio_features(uri)
    if audio_features:
        audio_features_dict[uri] = audio_features
        
# I need to convert dictionary into dataframe with uri as the first column
audio_features_dict = pd.DataFrame.from_dict(audio_features_dict, orient='index')
audio_features_dict.insert(0, 'uri', audio_features_dict.index)
audio_features_dict.reset_index(inplace=True, drop=True)

audio_features_dict.head(5)


### EDA & Cleaning of Additional Dataset

In [ ]:
audio_features_dict.isnull().sum()

In [ ]:
audio_features_dict.head(5)

In [ ]:
audio_features_dict.dtypes

I only want to assign one Key to the key value, so when there are enharmonic equivalents such as in key value 1, I will for the purpose of my analysis just use the sharp keys

pitch_class_dict = {
    0: 'C',
    1: 'C♯, D♭',
    2: 'D',
    3: 'D♯, E♭',
    4: 'E',
    5: 'F',
    6: 'F♯, G♭',
    7: 'G',
    8: 'G♯, A♭',
    9: 'A',
    10: 'A♯, B♭',
    11: 'B'
}

In [ ]:
pitch_class_dict = {
    0: 'C',
    1: 'C♯',
    2: 'D',
    3: 'D♯',
    4: 'E',
    5: 'F',
    6: 'F♯',
    7: 'G',
    8: 'G♯',
    9: 'A',
    10: 'A♯',
    11: 'B'
}

audio_features_dict['key'] = audio_features_dict['key'].map(pitch_class_dict)

The time_signature column represents the notational convention to specify how many beats are in each bar.  The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4". I want to change these numercial values to the actual time signatures eg 4 = 4/4 for ease of readability.

In [ ]:
time_signature_dict = {
    3: '3/4',
    4: '4/4',
    5: '5/4',
    6: '6/4',
    7: '7/4'
}
audio_features_dict['time_signature'] = audio_features_dict['time_signature'].map(time_signature_dict)

The mode column indicates the modality (major or minor) of a track and in the data from Spotify major is represented by 1 and minor is 0. I want to change the numercial encoding to the catergory names. 

In [ ]:
audio_features_dict['mode'] = audio_features_dict['mode'].replace({0:'minor', 1:'major'})

In [ ]:
audio_features_dict.head(5)

### Saving the Audio Features Dataframe

In [ ]:
# Specify the file path and name for the CSV file
file_path = "/Users/DataAnalyst/Desktop/track_audio_features.csv"

# Save the DataFrame as a CSV file
audio_features_dict.to_csv(file_path, index=False)

In [ ]:
audio_features_dict.shape

In [ ]:
audio_features_dict.dtypes

### Using Spotify's API to get dataframe for song duration & release date

In [ ]:
def get_track_info(uri):
    url = f'{base_url}tracks/{uri}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        track_info = response.json()
        selected_info = {'duration_ms': track_info['duration_ms'],
                        'release_date': track_info['album']['release_date']}
        return selected_info
    else:
        print(f"Error getting track_info for URI '{uri}': {response.content, response.status_code}")
        return None

# Dictionary to store the selected audio features for each URI
track_info_dict = {}

# Loop to iterate over each URI in the df_uris DataFrame
for uri in df_uris['uri']:
    track_info = get_track_info(uri)
    if track_info:
        track_info_dict[uri] = track_info
    
# convert dictionary into dataframe with uri as the first column
track_info_dict = pd.DataFrame.from_dict(track_info_dict, orient='index')
track_info_dict.insert(0, 'uri', track_info_dict.index)
track_info_dict.reset_index(inplace=True, drop=True)

track_info_dict.head(20)

In [ ]:
file_path = "/Users/DataAnalyst/Desktop/track_year_duration.csv"

track_info_dict.to_csv(file_path, index=False)

### Using Spotify's API to get dataframe for external URLs 

these URLs will be used as embedded links in my dashboard to play previews of the selected songs.

In [ ]:
def get_link_info(uri):
    url = f'{base_url}tracks/{uri}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        link_info = response.json()
        link_info = {'Spotify URL': link_info['external_urls']}
        return link_info
    else:
        print(f"Error getting track_info for URI '{uri}': {response.content, response.status_code}")
        return None

# Dictionary to store the selected audio features for each URI
link_info_dict = {}

# Loop to iterate over each URI in the df_uris DataFrame
for uri in df_uris['uri']:
    link_info = get_link_info(uri)
    if link_info:
        link_info_dict[uri] = link_info
    
# Convert dictionary into dataframe with uri as the first column
link_info_dict = pd.DataFrame.from_dict(link_info_dict, orient='index')
link_info_dict.insert(0, 'uri', link_info_dict.index)
link_info_dict.reset_index(inplace=True, drop=True)

link_info_dict.head(20)

### Merging the dataframes

In [ ]:
track_info_dict = pd.read_csv(r'/Users/DataAnalyst/Desktop/Spotify Data For Tableau/track_info(Year & Duration).csv')

In [ ]:
merged_df = pd.merge(track_info_dict, link_info_dict, on='uri', how='inner')
merged_df.rename(columns={'Spotify URL': 'url'}, inplace=True)
merged_df['url'] = merged_df['url'].astype(str)
merged_df['url'] = merged_df['url'].str.extract(r'(h.*).{2}$')
merged_df['url'] = merged_df['url'].str.replace('com', 'com/embed')
pd.set_option('display.max_colwidth', None)
merged_df.head(5)

In [ ]:
file_path = "/Users/DataAnalyst/Desktop/track_info_url.csv"

merged_df.to_csv(file_path, index=False)

In [ ]:
merged_df = df.merge(audio_features_dict, on='uri')

I needed to create a column with the Track Name & Artist name as the uri is different for the same tracks that are released as a single vs the album version. So this new column will be used as the unique identifier instead of the uri column.

In [ ]:
merged_df['track_artist_name'] = merged_df['track_name'] + "--" + merged_df['artist_name']

In [ ]:
df_new = pd.read_csv(r'/Users/DataAnalyst/Desktop/Spotify Data For Tableau/Spotify Listening History 2019 - 2022.csv')                                 

In [ ]:
df_new['track_artist_name'] = df_new['track_name'] + "--" + df_new['artist_name']

In [ ]:
df_new.to_csv(r'/Users/DataAnalyst/Desktop/Spotify Data For Tableau/Spotify Listening History 2019 - 2022.csv', index=False)

### Adjusting Audio Features Data

For each unique identifier (Track Name & Artist Name) I need to filter out a unique value for each audio feature, as when creating the dashboard I found that the Spotify algorithm had assigned multiple values for certain audio features for the same song eg. Gvae 2 tempo values for the same Track Name & Artist Name. 

The code belowe assigns the orginial value of the audio feature if there is only one unique value. Otherwise, it assigns the minimum audio feature value for each Track Name & Artist Name.

In [ ]:
# calculated the minimum audio feature for each unique track_artist_name
min_acousticness_by_artist = merged_df.groupby('track_artist_name')['acousticness'].transform('min')

# used boolean indexing to set adjusted column based on track_artist_name uniqueness
merged_df['Acousticness_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                              merged_df['acousticness'],
                                              min_acousticness_by_artist)

min_speechiness_by_artist = merged_df.groupby('track_artist_name')['speechiness'].transform('min')

merged_df['Speechiness_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                             merged_df['speechiness'],
                                             min_speechiness_by_artist)

min_liveness_by_artist = merged_df.groupby('track_artist_name')['liveness'].transform('min')

merged_df['Liveness_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                          merged_df['liveness'],
                                          min_liveness_by_artist)

min_energy_by_artist = merged_df.groupby('track_artist_name')['energy'].transform('min')

merged_df['Energy_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                        merged_df['energy'],
                                        min_energy_by_artist)

min_danceability_by_artist = merged_df.groupby('track_artist_name')['danceability'].transform('min')

merged_df['Danceability_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                              merged_df['danceability'],
                                              min_danceability_by_artist)

min_valence_by_artist = merged_df.groupby('track_artist_name')['valence'].transform('min')

merged_df['Valence_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                         merged_df['valence'],
                                         min_valence_by_artist)

min_instrumentalness_by_artist = merged_df.groupby('track_artist_name')['instrumentalness'].transform('min')

merged_df['Instrumentalness_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                         merged_df['instrumentalness'],
                                         min_instrumentalness_by_artist)

min_tempo_by_artist = merged_df.groupby('track_artist_name')['tempo'].transform('min')

merged_df['Tempo_Adjusted'] = np.where(merged_df['track_artist_name'].nunique() < 2,
                                         merged_df['tempo'],
                                         min_tempo_by_artist)

Now I need to drop the old columns

In [ ]:
merged_df=merged_df.drop(columns=['acousticness',
                                  'speechiness',
                                  'liveness',
                                  'energy',
                                  'danceability', 
                                  'valence', 
                                  'instrumentalness',
                                  'tempo'])

Now I can drop the columns I do not need, as I will be joining these datasets in Tableau with the unique identifier track_artist_name column

In [ ]:
column_number = merged_df.columns.get_loc('key')
merged_df = merged_df.iloc[:, column_number:]

In [ ]:
merged_df = merged_df.drop_duplicates(subset='track_artist_name', keep='first')

Now I can save this dataset. I will join all the the saved databases from this code using relationships in the Tableau data model.

In [ ]:
file_path = "/Users/DataAnalyst/Desktop/merged_audio_features.csv"

merged_df.to_csv(file_path, index=False)